In [1]:
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
import scipy.spatial.distance as dist
from scipy.stats import chi2_contingency
from sympy import *

In [2]:
#Code to disable scrolling

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Gini Index

In [4]:
def gini_index(df):
    total_rows = df.shape[0]
    print(f'Total Count: {total_rows}\n')
    attributes = list(df.columns[:-1])
    class_var = df.columns[-1]
    attr_gini = []
    for attr in attributes:
        print(f'\nAttribute: {attr}\n')
        value_counts = df[attr].value_counts()
        print(value_counts.to_string())
        values = value_counts.index
        gini_list =[]
        print('---------------------------')
        for value in values:
            print(f'\n{attr} = {value}\n')
            value_count = value_counts[value]
            class_count = df[df[attr] == value].groupby(class_var)[attr].count()
            print(class_count.to_string())
            print(f'\nTotal Count: {value_count}\n')
            gini = round(1-sum((class_count.values/value_count)**2),4)
            print_str = f'Gini = 1 - '
            for cnt in class_count.values:
                print_str += f'({cnt}/{value_count})\u00B2 - '
            print_str = print_str[:-2]
            print_str += f'= {gini}\n'
            print(print_str)
            gini_list.append(gini)
            print('---------------------------')
        
        total_gini = round(sum(np.array(gini_list)* np.array(value_counts.values)/total_rows),4)
        print_tot_str = f'For {attr}, Gini index = '
        for i in range(len(gini_list)):
            print_tot_str += f'{value_counts.values[i]}/{total_rows}*{gini_list[i]} + '
        print_tot_str = print_tot_str[:-2]
        print_tot_str += f'= {total_gini}\n'
        print(print_tot_str)
        attr_gini.append(total_gini)
        print('=================================================================================')
    print(f'\nFor {attributes} Gini index are {attr_gini}')
    print(f'\nMinimum Gini Index is for {attributes[attr_gini.index(min(attr_gini))]}')
    return attributes[attr_gini.index(min(attr_gini))]

In [5]:
df = pd.DataFrame({
                   'Income': ['High', 'Low', 'High', 'Middle', 'Low', 'High',
                               'Low', 'Low', 'Middle', 'Middle'],
                   'Education': ['Low', 'High', 'Low', 'High', 'Low', 'Low',
                               'High', 'Low', 'Low', 'High'],
                    'Gender': ['Male', 'Female', 'Male', 'Male', 'Male', 'Female',
                                                   'Male', 'Male', 'Female', 'Male'],
                    
                    'Buys Gadget': ['Yes', 'No', 'Yes', 'Yes', 'No', 'No',
                                                   'No', 'No', 'No', 'Yes'],})


In [6]:
df = pd.DataFrame({
                   'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy',
                               'Overcast', 'Sunny', 'Sunny', 'Rainy', 'Sunny', 'Overcast', 'Overcast',
                               'Rainy'],
                   'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool',
                               'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot',
                               'Mild'],
                    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal',
                                                   'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal',
                                                   'High'],
                    'Windy': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong',
                                                   'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak',
                                                   'Strong'],
                    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No',
                                                   'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
                                                   'No'],})

In [7]:
df

,Outlook,Temperature,Humidity,Windy,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rainy,Mild,High,Weak,Yes
4,Rainy,Cool,Normal,Weak,Yes
5,Rainy,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rainy,Mild,Normal,Weak,Yes


In [8]:
gini_index(df)

Total Count: 14


Attribute: Outlook

Sunny       5
Rainy       5
Overcast    4
---------------------------

Outlook = Sunny

PlayTennis
No     3
Yes    2

Total Count: 5

Gini = 1 - (3/5)² - (2/5)² = 0.48

---------------------------

Outlook = Rainy

PlayTennis
No     2
Yes    3

Total Count: 5

Gini = 1 - (2/5)² - (3/5)² = 0.48

---------------------------

Outlook = Overcast

PlayTennis
Yes    4

Total Count: 4

Gini = 1 - (4/4)² = 0.0

---------------------------
For Outlook, Gini index = 5/14*0.48 + 5/14*0.48 + 4/14*0.0 = 0.3429


Attribute: Temperature

Mild    6
Hot     4
Cool    4
---------------------------

Temperature = Mild

PlayTennis
No     2
Yes    4

Total Count: 6

Gini = 1 - (2/6)² - (4/6)² = 0.4444

---------------------------

Temperature = Hot

PlayTennis
No     2
Yes    2

Total Count: 4

Gini = 1 - (2/4)² - (2/4)² = 0.5

---------------------------

Temperature = Cool

PlayTennis
No     1
Yes    3

Total Count: 4

Gini = 1 - (1/4)² - (3/4)² = 0.375

------------

'Outlook'

## Entropy, Informaton Gain and Gain ratio

In [9]:
def information_gain(df, P = 'Yes', N = 'No'):
    total_rows = df.shape[0]
    print(f'Total Count: {total_rows}\n')
    
    attributes = list(df.columns[:-1])
    class_var = df.columns[-1]
    df_val_count = df[class_var].value_counts()
    print(df_val_count.to_string())
    P_count = df_val_count[P]
    N_count = df_val_count[N]
    print(f'\nP: {P_count}, N: {N_count}\n')
    entropy_s = -P_count/(P_count + N_count)*np.log2(P_count/(P_count + N_count)) -N_count/(P_count + N_count)*np.log2(N_count/(P_count + N_count))
    entropy_s = round(entropy_s, 4)
    print(f'\nEntropy(S) = -{P_count}/({P_count} + {N_count})*log({P_count}/({P_count} + {N_count})) -{N_count}/({P_count} + {N_count})*log({N_count}/({P_count} + {N_count})) = {entropy_s}')
    attr_information_gain = []
    attr_gain_ratio = []
    for attr in attributes:
        print(f'\nAttribute: {attr}\n')
        value_counts = df[attr].value_counts()
        print(value_counts.to_string())
        values = value_counts.index
        entropy_list =[]
        information_list = []
        information_str = ''
        print('---------------------------')
        for value in values:
            print(f'\n{attr} = {value}\n')
            value_count = value_counts[value]
            class_count = df[df[attr] == value].groupby(class_var)[attr].count()
            print(class_count.to_string())
            print(f'\nTotal Count: {value_count}\n')
            try:
                P_i_count = class_count[P]
            except:
                P_i_count = 0
            try:
                N_i_count = class_count[N]
            except:
                N_i_count = 0
            
            print(f'\nP: {P_i_count}, N: {N_i_count}\n')
            if P_i_count ==0 or N_i_count ==0:
                entropy_i = 0
            else:
                entropy_i = -P_i_count/(P_i_count + N_i_count)*np.log2(P_i_count/(P_i_count + N_i_count)) -N_i_count/(P_i_count + N_i_count)*np.log2(N_i_count/(P_i_count + N_i_count))
            entropy_i = round(entropy_i, 4)
            print(f'\nEntropy({attr} = {value}) = -{P_i_count}/({P_i_count} + {N_i_count})*log({P_i_count}/({P_i_count} + {N_i_count})) -{N_i_count}/({P_i_count} + {N_i_count})*log({N_i_count}/({P_i_count} + {N_i_count})) = {entropy_i}')
            information_i = round((P_i_count+N_i_count)/(P_count + N_count)*entropy_i,4)
            information_list.append(information_i)
            information_str += f'({P_i_count}+{N_i_count})/({P_count} + {N_count})*{entropy_i} + '
            entropy_list.append(entropy_i)
            print('---------------------------')
        information_str = information_str[:-2]
        average_information = round(sum(information_list),4)
        print(f'\nAverage Information({attr}) = {information_str} = {average_information}')
        info_gain = round(entropy_s - average_information,4)
        split_info = round(sum([-x/total_rows*np.log2(x/total_rows) for x in value_counts]),4)
        split_info_str = [f'{-x}/{total_rows}*log({x}/{total_rows})' for x in value_counts]
       
        print(f'\nInformation Gain({attr}) = Entropy(S) - Average Information({attr})', end='')
        print(f' = {entropy_s} - {average_information} = {info_gain}\n')
        print(f'Split info({attr}) = {" ".join(split_info_str)} = {round(split_info,4)}')
        gain_ratio = round(info_gain/split_info,4)
        print(f'\nGain ratio({attr}) = Information Gain({attr})/Split info({attr}) = {info_gain}/{split_info} = {gain_ratio}')
        attr_information_gain.append(info_gain)
        attr_gain_ratio.append(gain_ratio)
        print('\n=================================================================================')
        
    print(f'\nFor {attributes} Information gain are {attr_information_gain} and gain ratios are {attr_gain_ratio}')
    print(f'Maximum Information Gain is for {attributes[attr_information_gain.index(max(attr_information_gain))]}')
    print(f'Maximum Gain ratio is for {attributes[attr_gain_ratio.index(max(attr_gain_ratio))]}')
    return attributes[attr_gain_ratio.index(max(attr_gain_ratio))]
    

In [11]:
df = pd.DataFrame({
                   'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Rainy',
                               'Overcast', 'Sunny', 'Sunny', 'Rainy', 'Sunny', 'Overcast', 'Overcast',
                               'Rainy'],
                   'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool',
                               'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot',
                               'Mild'],
                    'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal',
                                                   'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal',
                                                   'High'],
                    'Windy': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong',
                                                   'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak',
                                                   'Strong'],
                    'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No',
                                                   'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes',
                                                   'No'],})
information_gain(df)

Total Count: 14

Yes    9
No     5

P: 9, N: 5


Entropy(S) = -9/(9 + 5)*log(9/(9 + 5)) -5/(9 + 5)*log(5/(9 + 5)) = 0.9403

Attribute: Outlook

Sunny       5
Rainy       5
Overcast    4
---------------------------

Outlook = Sunny

PlayTennis
No     3
Yes    2

Total Count: 5


P: 2, N: 3


Entropy(Outlook = Sunny) = -2/(2 + 3)*log(2/(2 + 3)) -3/(2 + 3)*log(3/(2 + 3)) = 0.971
---------------------------

Outlook = Rainy

PlayTennis
No     2
Yes    3

Total Count: 5


P: 3, N: 2


Entropy(Outlook = Rainy) = -3/(3 + 2)*log(3/(3 + 2)) -2/(3 + 2)*log(2/(3 + 2)) = 0.971
---------------------------

Outlook = Overcast

PlayTennis
Yes    4

Total Count: 4


P: 4, N: 0


Entropy(Outlook = Overcast) = -4/(4 + 0)*log(4/(4 + 0)) -0/(4 + 0)*log(0/(4 + 0)) = 0
---------------------------

Average Information(Outlook) = (2+3)/(9 + 5)*0.971 + (3+2)/(9 + 5)*0.971 + (4+0)/(9 + 5)*0  = 0.6936

Information Gain(Outlook) = Entropy(S) - Average Information(Outlook) = 0.9403 - 0.6936 = 0.2467

Split info(Ou

'Outlook'

## Construct Tree

In [12]:
def construct_tree(df, method):
    print('Input table:')
    display(df)
    if method =='gini':
        print('\nConstructing tree based on gini\n')
        attr = gini_index(df)
    else:
        print('\nConstructing tree based entropy/Information gain\n')
        attr = information_gain(df)
    print('\n=================================================================================')
    print('\n=================================================================================')
    print(f'\nSplitting the table based on {attr}')
    value_counts = df[attr].value_counts()
    #print(value_counts.to_string())
    values = value_counts.index
    gini_list =[]

    for value in values:
        print(f'\n{attr} = {value}\n')
        
        df_attr = df[df[attr] == value]
        
        val_cnt = df_attr[df_attr.columns[-1]].value_counts()
        if val_cnt.shape[0] == 1:
            print('Input table:')
            display(df_attr)
            print(f'\nFor {attr} = {value} we have only one class label, so no need for further splitting\n')
        else:
            construct_tree(df_attr.drop(columns = [attr]), method)

In [13]:
construct_tree(df, method = 'entropy')

Input table:


,Outlook,Temperature,Humidity,Windy,PlayTennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rainy,Mild,High,Weak,Yes
4,Rainy,Cool,Normal,Weak,Yes
5,Rainy,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rainy,Mild,Normal,Weak,Yes



Constructing tree based entropy/Information gain

Total Count: 14

Yes    9
No     5

P: 9, N: 5


Entropy(S) = -9/(9 + 5)*log(9/(9 + 5)) -5/(9 + 5)*log(5/(9 + 5)) = 0.9403

Attribute: Outlook

Sunny       5
Rainy       5
Overcast    4
---------------------------

Outlook = Sunny

PlayTennis
No     3
Yes    2

Total Count: 5


P: 2, N: 3


Entropy(Outlook = Sunny) = -2/(2 + 3)*log(2/(2 + 3)) -3/(2 + 3)*log(3/(2 + 3)) = 0.971
---------------------------

Outlook = Rainy

PlayTennis
No     2
Yes    3

Total Count: 5


P: 3, N: 2


Entropy(Outlook = Rainy) = -3/(3 + 2)*log(3/(3 + 2)) -2/(3 + 2)*log(2/(3 + 2)) = 0.971
---------------------------

Outlook = Overcast

PlayTennis
Yes    4

Total Count: 4


P: 4, N: 0


Entropy(Outlook = Overcast) = -4/(4 + 0)*log(4/(4 + 0)) -0/(4 + 0)*log(0/(4 + 0)) = 0
---------------------------

Average Information(Outlook) = (2+3)/(9 + 5)*0.971 + (3+2)/(9 + 5)*0.971 + (4+0)/(9 + 5)*0  = 0.6936

Information Gain(Outlook) = Entropy(S) - Average Information

,Temperature,Humidity,Windy,PlayTennis
0,Hot,High,Weak,No
1,Hot,High,Strong,No
7,Mild,High,Weak,No
8,Cool,Normal,Weak,Yes
10,Mild,Normal,Strong,Yes



Constructing tree based entropy/Information gain

Total Count: 5

No     3
Yes    2

P: 2, N: 3


Entropy(S) = -2/(2 + 3)*log(2/(2 + 3)) -3/(2 + 3)*log(3/(2 + 3)) = 0.971

Attribute: Temperature

Hot     2
Mild    2
Cool    1
---------------------------

Temperature = Hot

PlayTennis
No    2

Total Count: 2


P: 0, N: 2


Entropy(Temperature = Hot) = -0/(0 + 2)*log(0/(0 + 2)) -2/(0 + 2)*log(2/(0 + 2)) = 0
---------------------------

Temperature = Mild

PlayTennis
No     1
Yes    1

Total Count: 2


P: 1, N: 1


Entropy(Temperature = Mild) = -1/(1 + 1)*log(1/(1 + 1)) -1/(1 + 1)*log(1/(1 + 1)) = 1.0
---------------------------

Temperature = Cool

PlayTennis
Yes    1

Total Count: 1


P: 1, N: 0


Entropy(Temperature = Cool) = -1/(1 + 0)*log(1/(1 + 0)) -0/(1 + 0)*log(0/(1 + 0)) = 0
---------------------------

Average Information(Temperature) = (0+2)/(2 + 3)*0 + (1+1)/(2 + 3)*1.0 + (1+0)/(2 + 3)*0  = 0.4

Information Gain(Temperature) = Entropy(S) - Average Information(Temperature) = 0

...


For Windy = Weak we have only one class label, so no need for further splitting


Windy = Strong

Input table:


,Temperature,Humidity,Windy,PlayTennis
5,Cool,Normal,Strong,No
13,Mild,High,Strong,No



For Windy = Strong we have only one class label, so no need for further splitting


Outlook = Overcast

Input table:


,Outlook,Temperature,Humidity,Windy,PlayTennis
2,Overcast,Hot,High,Weak,Yes
6,Overcast,Cool,Normal,Strong,Yes
11,Overcast,Mild,High,Strong,Yes
12,Overcast,Hot,Normal,Weak,Yes



For Outlook = Overcast we have only one class label, so no need for further splitting

